<a href="https://colab.research.google.com/github/MikeG27/colab_backups/blob/main/Agentic_AI_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crew.ai - Create your own Crew of Agents!

Welcome to the fascinating world of CrewAI! Today, we're diving into an exciting exploration where we'll experiment with both **single-agent** and **multi-agent** flows. You'll see how individual agents work independently and how combining multiple agents in one cohesive process can revolutionize automation, data analysis, and content generation.


## Overview

crewAI is a powerful framework designed for orchestrating multiple AI agents to work collaboratively on complex tasks. It allows developers to define, manage, and execute structured workflows involving different roles and responsibilities among AI agents.


## Why CrewAI?
What makes **CrewAI** stand out is its **simplicity and ease of use**. Despite its powerful capabilities in orchestrating multiple AI agents, the framework is designed to be **intuitive and developer-friendly**. Whether you're working with a single agent or building a complex multi-agent workflow, CrewAI provides a structured yet flexible way to automate tasks, making it accessible even for those new to AI automation.


## Main Abstractions in CrewAI
CrewAI operates around a structured set of abstractions that define how AI agents interact:

* Agent – A single AI entity with a role, tools, and capabilities (e.g., a "Researcher" or "Writer").
* Task – A specific job assigned to an agent (e.g., "Summarize an article" or "Extract key data").
* Crew – A collection of agents working together on related tasks.
* Process – A sequence of actions performed by agents to achieve a goal.
* Tools – APIs, search engines, or databases that an agent can use for information retrieval and execution.

## Important notes

1. **Please be aware that using the OpenAI API incurs costs.** Since the agents generate multiple API calls to produce the final outputs, we are currently using a lightweight reasoning mode (gpt4-o-mini) to keep expenses in check. If you decide to switch to a model with enhanced reasoning capabilities, please proceed at your own risk as this may lead to increased costs.
2. Final results may vary due to the stochastic nature of AI models. Since these examples are for demonstration purposes only, the final output quality might not always meet the highest standards.
3. After reading through this guide, I encourage you to explore the examples, experiment with the code, and **create something simple but uniquely your own.** Playing around with different configurations and agents is the best way to truly understand how CrewAI works!




# Get Started

In this notebook we will do 3 projects.
1. Single agent with websearch
2. Multi-agent writing workflow
3. Multi Agent for financial analysis

## Install Dependencies

Before building our agent, we need to install the required dependencies.
The following commands install CrewAI, LangChain, and additional tools necessary for building AI agents, task automation, and integration with external APIs.

Runtime Consideration:
To ensure smooth execution, select a T4 GPU as the runtime environment in Google Colab or your preferred setup. After installation, you may need to restart the runtime for the changes to take effect. (just for the step below)

Commentary:
These commands ensure that all necessary libraries are installed. The installation of 'crewai[tools]' includes optional tool support.

In [ ]:
!pip install -U crewai==0.28.8 crewai_tools==0.1.6 langchain_community
!pip install 'crewai[tools]'

## Import dependencies

Now that our environment is set up, we need to import the necessary libraries to build our weather forecast agent:

* warnings and os: For handling warnings and operating system interactions.
* google.colab.userdata: To securely retrieve API keys.
* CrewAI components: Such as Agent, Task, Crew, and LLM.
* CrewAI Tools: For example, ScrapeWebsiteTool and SerperDevTool.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
from google.colab import userdata
from crewai import Agent, Task, Crew # import crew components
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from IPython.display import Markdown

## Setup API Keys

Environment variables are set for API keys required by OpenAI and Serper:

* OpenAI API Key: For Azure’s OpenAI endpoints.
* Serper API Key: For a fast and cost-effective Google search API.

[Here](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) you have quick tutorial how to set up keys in colab environment

In [ ]:
import os
from google.colab import userdata

os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://saturn-poc1.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"

### Setting Up the LLM
The LLM is instantiated with a specific model:

In [ ]:
# Use AzureOpenAI for chat-based models (gpt-4-turbo)
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4o-mini",
    temperature=0
)

# Example 1: Single Agent with Web Search


**Objective:**
- Create an AI agent that fetches real-time weather forecasts for a given location.

Steps

1. Define your own tool to retrieve weather data using Open-Meteo API and a geolocation API for latitude/longitude lookup.
2. Create an agent that specializes in providing weather forecasts.
3. Assign a task to fetch and summarize weather data.
Execute the agent’s workflow.


### Tool Definition: get_weather_forecast
A tool is defined using a decorator to fetch weather information:

Functionality:
1. Geocoding: Converts a location string into latitude and longitude using the Photon API.
2. Weather Data: Fetches weather details (temperature, humidity, wind speed) from the Open-Meteo API.

In [ ]:
import requests
from crewai_tools import tool

@tool("Weather Tool")
def get_weather_forecast(location: str):
    """
    Retrieves the weather forecast for a given location.

    Args:
        location (str): The name of the location (e.g., "London").

    Returns:
        dict: A dictionary containing weather information, or None if an error occurs.
    """
    try:
        # Geocoding to get latitude and longitude
        geocoding_api_url = f"https://photon.komoot.io/api/?q={location}"
        geocoding_response = requests.get(geocoding_api_url)
        geocoding_response.raise_for_status()
        geocoding_data = geocoding_response.json()

        if geocoding_data and geocoding_data.get("features"):
            latitude = float(geocoding_data["features"][0]["geometry"]["coordinates"][1])
            longitude = float(geocoding_data["features"][0]["geometry"]["coordinates"][0])
        else:
            print(f"Location '{location}' not found.")
            return None

        # Fetching weather data using Open-Meteo API
        weather_api_url = (
            f"https://api.open-meteo.com/v1/forecast?"
            f"latitude={latitude}&longitude={longitude}&hourly=temperature_2m,relativehumidity_2m,windspeed_10m"
        )
        weather_response = requests.get(weather_api_url)
        weather_response.raise_for_status()
        weather_data = weather_response.json()

        # Extracting relevant weather information
        forecast = {
            "location": location,
            "latitude": latitude,
            "longitude": longitude,
            "temperature": weather_data.get("hourly", {}).get("temperature_2m", []),
            "humidity": weather_data.get("hourly", {}).get("relativehumidity_2m", []),
            "wind_speed": weather_data.get("hourly", {}).get("windspeed_10m", [])
        }

        return forecast

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None


### Weather Agent and Task
A Weather Expert Agent is created:

* Role: "Weather Expert"
* Goal: Provide weather forecasts.
* Backstory: Context such as "An experienced meteorologist."
* Tools: Uses the get_weather_forecast tool.
* LLM: Uses a detailed LLM model (azure/gpt-4o-standard).

In [ ]:
weather_agent = Agent(
    role='Weather Expert',
    goal='Provide weather forecasts for given locations',
    backstory='An experienced meteorologist.',
    tools=[get_weather_forecast],
    llm=llm,
    verbose=True
)

### Defining the Task

The agent is assigned a specific task to retrieve weather data for a location.

In [ ]:
task = Task(
    description=(
        """Get the current weather forecast for ({location})
        Provide a summary of the temperature, humidity, and wind speed."""
    ),
    expected_output="A beutifull weather forecast provided in plain text including temperature, humidity, and wind speed in ({location}).",
    agent=weather_agent
)


crew = Crew(
    agents=[weather_agent],
    tasks=[task],
    verbose=True,
)


### Running the Agent

In [ ]:
# change location if you want
location_inputs = {
    'location': 'London'
}

result = crew.kickoff(inputs=location_inputs)
print(result)

 [DEBUG]: == Working Agent: Weather Expert
 [INFO]: == Starting Task: Get the current weather forecast for (London)
        Provide a summary of the temperature, humidity, and wind speed.


> Entering new CrewAgentExecutor chain...
I need to retrieve the current weather forecast for London to provide the necessary details about temperature, humidity, and wind speed.

Action: Weather Tool  
Action Input: {"location": "London"}   

{'location': 'London', 'latitude': 51.5074456, 'longitude': -0.1277653, 'temperature': [3.6, 3.3, 2.9, 2.2, 2.1, 1.8, 1.6, 1.7, 2.4, 3.6, 4.7, 6.3, 7.2, 7.9, 8.6, 8.5, 7.7, 6.8, 6.6, 5.8, 5.8, 5.6, 5.4, 4.9, 4.3, 3.8, 3.3, 3.1, 2.9, 2.5, 2.2, 2.3, 2.7, 3.6, 5.3, 6.7, 7.2, 7.3, 7.2, 7.4, 7.5, 7.0, 6.3, 5.3, 4.5, 3.7, 3.1, 2.6, 2.1, 1.8, 1.6, 1.4, 1.4, 1.3, 1.4, 0.4, 1.4, 3.0, 5.0, 7.0, 8.1, 9.0, 9.3, 9.3, 9.4, 8.6, 7.9, 7.2, 6.8, 6.5, 5.9, 5.4, 4.9, 4.8, 4.3, 3.8, 3.7, 3.6, 3.7, 4.3, 5.2, 6.0, 6.5, 6.9, 7.3, 7.6, 8.0, 8.0, 7.7, 7.0, 6.4, 5.9, 5.4, 4.9, 4.5, 4.2

### Summary:
This section demonstrates a single-agent flow where a weather agent uses a custom tool to fetch weather data, processes it, and presents the forecast. It illustrates how tools and tasks are integrated within the CrewAI framework.

# Example 2: Writing Agent (Multi-step Content Creation)
This example demonstrates a multi-agent flow involving:

* Content Planner: Plans the article.
* Content Writer: Writes the blog post using the plan.
* Editor: Reviews and refines the content.

###Agent Definitions
Each agent is instantiated with a role, goal, and backstory. Placeholders like {topic} are used to create dynamic tasks.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm,
	verbose=True
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Task Definitions
Tasks are assigned to each agent:

* Plan Task: Outlines the content, target audience, and SEO keywords.
* Write Task: Crafts the blog post based on the plan.
* Edit Task: Proofreads and refines the content.


In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

The agents and tasks are then grouped into a crew:


In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [ ]:
article = crew.kickoff(inputs={"topic": "Gravitee.io"})
#PS we use here internal knowledge of the model itself, not neccesarely info from web.

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Gravitee.io.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.  
Final Answer: 

**Content Plan for Blog Article on Gravitee.io**

---

### 1. Latest Trends, Key Players, and Noteworthy News on Gravitee.io

**Latest Trends:**
- **API Management Growth:** The demand for API management solutions is increasing as businesses shift towards microservices and cloud-native architectures. Gravitee.io is positioned well in this trend with its open-source API management platform.
- **Focus on Security:** With the rise in cyber threats, API security has become a top priority. Gravitee.io offers robust securit

In [ ]:
with open("article.md", "w", encoding="utf-8") as f:
    f.write(article)

### Summary:
This section showcases a sequential multi-agent flow where content is planned, written, and then edited. **It highlights how different agents with specialized roles work together to produce a final article.**

# Example 3: Multi-Agent for Financial Analysis
This example outlines a hierarchical multi-agent system for financial analysis and trading recommendations.

### Tools & LLM Setup
Instantiate specialized tools:

* SerperDevTool: For fast Google search.
* ScrapeWebsiteTool: For scraping website content.


In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
# Use AzureOpenAI for chat-based models (gpt-4-turbo)
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4o-standard",
    temperature=0
)

### Agent Definitions
Four specialized agents are created:

1. Data Analyst Agent: Monitors and analyzes market data.
2. Trading Strategy Developer: Develops trading strategies.
3. Trade Advisor: Suggests trade execution strategies.
4. Risk Advisor: Assesses trading risks.

In [ ]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

### Task Definitions
Tasks corresponding to each agent:

1. Data Analysis Task: Analyzes market data.
2. Strategy Development Task: Crafts trading strategies.
3. Execution Planning Task: Plans optimal trade execution.
4. Risk Assessment Task: Analyzes potential trading risks.

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for  "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

### Manager and Crew Setup
A Project Manager agent is introduced to oversee the tasks in a hierarchical process:

In [ ]:
# Define the manager agent for tasks delegation
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm,
    verbose=True
)

### Assemble the crew with all agents, tasks, and a hierarchical process:


In [ ]:
from crewai import Crew, Process


# Define the crew with agents and tasks
# execution agent and risk_agent removed to limit complexity. IF you want feel free to use
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
           # execution_agent,
           # risk_management_agent
            ],

    tasks=[data_analysis_task,
           strategy_development_task,
          #  execution_planning_task,
          # risk_assessment_task
           ],

    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

In [ ]:
### this execution will take some time to run

financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': False
}
trading_recomendations = financial_trading_crew.kickoff(inputs=financial_trading_inputs)
trading_markdown_content = trading_recomendations
Markdown(trading_markdown_content)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for  the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
To complete the task of continuously monitoring and analyzing market data for AAPL, and using statistical modeling and machine learning to identify trends and predict market movements, I need to delegate specific tasks to the appropriate co-workers and ask relevant questions to gather all necessary information.

First, I will delegate the task of collecting and analyzing the market data for AAPL to the Data Analyst. This will include gathering historical data, current market data, and any relevant financial indicators.

Thought: I need to delegate the task of collecting and analyzing market data for AAPL to the Data Analyst.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "C

Based on the provided insights and historical data, here are the refined trading strategies for AAPL that align with a medium risk tolerance and a preference for day trading:

1. **Momentum Trading**:
   - **Strategy**: Buy AAPL stock as it shows upward momentum and sell when the momentum slows down.
   - **Execution**: Monitor key momentum indicators such as Moving Average Convergence Divergence (MACD) and Relative Strength Index (RSI). Enter trades when the MACD line crosses above the signal line and RSI is above 50. Exit trades when the MACD line crosses below the signal line or RSI drops below 50.

2. **Breakout Trading**:
   - **Strategy**: Buy AAPL stock when it breaks above a significant resistance level and sell when it breaks below a significant support level.
   - **Execution**: Identify key resistance and support levels using historical price data. Enter trades when the stock price breaks above resistance with high volume. Exit trades when the stock price breaks below support with high volume.

3. **Mean Reversion Trading**:
   - **Strategy**: Buy AAPL stock when it deviates significantly below its average price and sell when it reverts to the mean.
   - **Execution**: Use Bollinger Bands to identify overbought and oversold conditions. Enter trades when the stock price touches the lower Bollinger Band and RSI is below 30. Exit trades when the stock price reverts to the middle Bollinger Band or RSI is above 50.

4. **Scalping**:
   - **Strategy**: Make multiple small trades throughout the day to capture small price movements.
   - **Execution**: Use a 1-minute or 5-minute chart to identify short-term price patterns. Enter and exit trades quickly based on these patterns, aiming for small profits on each trade. Use tight stop-loss orders to minimize potential losses.

5. **Risk Management**:
   - **Position Sizing**: Allocate a fixed percentage of the portfolio to each trade to manage risk. For example, allocate 1-2% of the portfolio to each trade.
   - **Stop-Loss Orders**: Set stop-loss orders at key support levels to protect against significant losses. For example, set a stop-loss order 2-3% below the entry price for each trade.
   - **Take-Profit Orders**: Set take-profit orders at key resistance levels to lock in profits. For example, set a take-profit order 2-3% above the entry price for each trade.

By combining these strategies and continuously monitoring the market conditions, significant market opportunities or threats for AAPL can be identified and acted upon effectively.

In [ ]:
with open("Risk_analysis_report.md", "w", encoding="utf-8") as f:
    f.write(trading_markdown_content)

### Summary:
This section describes a hierarchical multi-agent system for financial analysis. It demonstrates the integration of specialized agents and tools, with a project manager coordinating the workflow to generate trade recommendations.

# Extras
1. CrewAI Docs: https://docs.crewai.com/introduction
2. Agents design patterns: https://www.anthropic.com/engineering/building-effective-agents